# BLIP 기반 랜드마크 이미지 인식 및 검증 시스템

3가지 검증 방식 제공:
1. 이미지-텍스트 유사도 검증 (BlipForImageTextRetrieval)
2. 텍스트-텍스트 유사도 검증 (캡션 생성 + SBERT)
3. VQA 유사도 검증 (질문-답변 기반)

In [ ]:
import torch
from transformers import (BlipProcessor,
                          BlipForImageTextRetrieval,
                          BlipForConditionalGeneration,
                          BlipForQuestionAnswering)
from PIL import Image
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import glob
import os
import io
import sys
from pathlib import Path
from tqdm.auto import tqdm

# 환경 감지
try:
    from google.colab import files, drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# GPU 사용 설정
device = "cuda" if torch.cuda.is_available() else "cpu"

# 전역 변수로 모델 캐시 (lazy loading 용)
_models_cache = {}

## 1. 데이터 디렉토리 설정

In [2]:
# ============================================================================
# 1. 데이터 디렉토리 설정
# ============================================================================

def setup_data_directory(custom_path=None):
    """
    데이터 디렉토리를 설정하는 함수
    
    Args:
        custom_path: 사용자 지정 경로 (None이면 자동 감지)
    
    Returns:
        data_dir: 설정된 데이터 디렉토리 경로
    """
    if custom_path:
        return custom_path
    
    if IN_COLAB:
        # Colab 환경: Google Drive 마운트 시도
        try:
            drive.mount('/content/drive/', force_remount=False)
            return "/content/drive/MyDrive/파주 AI 프로젝트/data"
        except Exception as e:
            print(f"Google Drive 마운트 실패: {e}")
            return "/content/data"
    else:
        # 로컬 환경: 상대 경로 사용
        current_file = Path(__file__).parent
        return str(current_file / "data")

# 기본 데이터 디렉토리 설정
DATA_DIR = setup_data_directory()

NameError: name 'IN_COLAB' is not defined

## 2. 모델 로드 (Lazy Loading)

In [ ]:
# ============================================================================
# 2. 모델 로드 (Lazy Loading)
# ============================================================================

@torch.no_grad()
def load_model(mode):
    """
    BLIP 모델을 로드하는 함수 (캐싱 지원)
    
    Args:
        mode: 모델 모드 ("captioning", "matching", "vqa")
    
    Returns:
        processor, model: 로드된 프로세서와 모델
    """
    # 이미 로드된 모델이 있으면 반환
    if mode in _models_cache:
        return _models_cache[mode]
    
    print(f"[{mode}] 모델을 로드하는 중... (장치: {device})")
    
    try:
        if mode == "captioning":
            model_name = "Salesforce/blip-image-captioning-base"
            processor = BlipProcessor.from_pretrained(model_name)
            model = BlipForConditionalGeneration.from_pretrained(model_name).to(device)
        elif mode == "matching":
            model_name = "Salesforce/blip-itm-base-coco"
            processor = BlipProcessor.from_pretrained(model_name)
            model = BlipForImageTextRetrieval.from_pretrained(model_name).to(device)
        elif mode == "vqa":
            model_name = "ybelkada/blip-vqa-base"
            processor = BlipProcessor.from_pretrained(model_name)
            model = BlipForQuestionAnswering.from_pretrained(model_name).to(device)
        else:
            raise ValueError(f"알 수 없는 모드: {mode}. 사용 가능한 모드: 'captioning', 'matching', 'vqa'")
        
        model.eval()
        _models_cache[mode] = (processor, model)
        print(f"[{mode}] 모델 로드 완료!")
        
        return processor, model
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")
        raise

def load_sbert_model():
    """SBERT 모델을 로드하는 함수 (캐싱 지원)"""
    if "sbert" not in _models_cache:
        print(f"SBERT 모델을 로드하는 중... (장치: {device})")
        _models_cache["sbert"] = SentenceTransformer("all-MiniLM-L6-v2", device=device)
        print("SBERT 모델 로드 완료!")
    return _models_cache["sbert"]

## 3. Ground Truth 캡션 정의 (랜드마크 이미지 폴더에서 자동 생성)

In [ ]:
# ============================================================================
# 3. Ground Truth 캡션 정의 (랜드마크 이미지 폴더에서 자동 생성)
# ============================================================================

def find_images_in_directory(directory):
    """
    디렉토리에서 이미지 파일을 찾는 함수 (대소문자 구분 없음)
    
    Args:
        directory: 이미지를 찾을 디렉토리 경로
    
    Returns:
        image_files: 찾은 이미지 파일 경로 리스트
    """
    image_extensions = ["jpg", "jpeg", "png", "webp", "jfif", "bmp", "gif"]
    image_files = []
    
    if not os.path.exists(directory):
        return []
    
    for file in os.listdir(directory):
        file_path = os.path.join(directory, file)
        if os.path.isfile(file_path):
            ext = file.lower().split('.')[-1]
            if ext in image_extensions:
                image_files.append(file_path)
    
    return image_files

In [ ]:
def generate_ground_truth_captions_from_folder(landmark_name, data_dir=None, prompt_text=None):
    """
    랜드마크 폴더의 모든 이미지에서 Ground Truth 캡션을 생성
    
    Args:
        landmark_name: 랜드마크 이름 (폴더 이름)
        data_dir: 데이터 디렉토리 경로 (None이면 기본값 사용)
        prompt_text: 선택적 프롬프트 텍스트 (예: "this place is")
    
    Returns:
        all_captions: 모든 이미지에서 생성된 고유 캡션 리스트
    """
    # 모델 로드
    cap_processor, cap_model = load_model("captioning")
    
    # 데이터 디렉토리 설정
    if data_dir is None:
        data_dir = DATA_DIR
    
    landmark_dir = os.path.join(data_dir, landmark_name)
    
    # 이미지 파일 찾기
    image_files = find_images_in_directory(landmark_dir)
    
    if not image_files:
        print(f"경고: '{landmark_dir}' 폴더에서 이미지를 찾을 수 없습니다.")
        print(f"데이터 디렉토리: {data_dir}")
        return []
    
    print(f"'{landmark_name}' 폴더에서 총 {len(image_files)}개의 이미지를 찾았습니다.")
    print("Ground Truth 캡션을 생성 중입니다...")
    
    all_captions_set = set()  # 중복 제거를 위한 set
    failed_images = 0
    
    for img_path in tqdm(image_files, desc=f"'{landmark_name}' 이미지 처리 중"):
        try:
            image = Image.open(img_path).convert("RGB")
            
            # 캡션 생성
            if prompt_text:
                inputs = cap_processor(images=image, text=prompt_text, return_tensors="pt").to(device)
            else:
                inputs = cap_processor(images=image, return_tensors="pt").to(device)
            
            # Greedy 디코딩으로 기본 캡션 생성
            outputs_greedy = cap_model.generate(**inputs, max_length=50)
            caption_greedy = cap_processor.decode(outputs_greedy[0], skip_special_tokens=True).strip()
            
            if caption_greedy and caption_greedy.lower() not in ["none", "unanswerable", ""]:
                all_captions_set.add(caption_greedy)
            
            # 다양한 캡션 생성 (Nucleus Sampling)
            outputs_diverse = cap_model.generate(
                **inputs,
                max_length=50,
                num_beams=5,
                do_sample=True,
                top_p=0.9,
                num_return_sequences=3
            )
            
            for output in outputs_diverse:
                caption = cap_processor.decode(output, skip_special_tokens=True).strip()
                if caption and caption.lower() not in ["none", "unanswerable", ""]:
                    all_captions_set.add(caption)
                    
        except Exception as e:
            failed_images += 1
            if failed_images <= 3:  # 처음 3개만 출력
                print(f"\n{os.path.basename(img_path)} 처리 중 오류 발생: {e}")
    
    all_captions = list(all_captions_set)
    print(f"\n생성된 Ground Truth 캡션 수: {len(all_captions)}")
    if failed_images > 0:
        print(f"처리 실패한 이미지 수: {failed_images}")
    
    return all_captions

In [ ]:
def generate_single_image_caption(image, prompt_text=None):
    """
    단일 이미지로부터 캡션 생성 (Ground Truth 생성용)
    
    Args:
        image: PIL Image 객체
        prompt_text: 선택적 프롬프트 텍스트
    
    Returns:
        generated_captions: 생성된 캡션 리스트
    """
    # 모델 로드
    cap_processor, cap_model = load_model("captioning")
    
    print("\n--- AI가 Ground Truth 캡션을 생성합니다 ---")
    
    try:
        if prompt_text:
            inputs = cap_processor(images=image, text=prompt_text, return_tensors="pt").to(device)
        else:
            inputs = cap_processor(images=image, return_tensors="pt").to(device)
        
        # 기본 캡션 생성
        outputs_greedy = cap_model.generate(**inputs, max_length=50)
        caption_greedy = cap_processor.decode(outputs_greedy[0], skip_special_tokens=True)
        print(f"  [기본 캡션]: {caption_greedy}")
        
        # 다양한 캡션 생성
        print("\n[다양한 캡션 (JSON 후보 3개)]:")
        outputs_diverse = cap_model.generate(
            **inputs,
            max_length=50,
            num_beams=5,
            do_sample=True,
            top_p=0.9,
            num_return_sequences=3
        )
        
        generated_captions = []
        for i, output in enumerate(outputs_diverse):
            caption = cap_processor.decode(output, skip_special_tokens=True)
            if caption not in generated_captions and caption != caption_greedy:
                generated_captions.append(caption)
            print(f"  - (후보 {i+1}): {caption}")
        
        if caption_greedy not in generated_captions:
            generated_captions.insert(0, caption_greedy)
        
        # JSON 형식으로 출력
        print("\n--- 'missions.json'의 [ground_truth_captions]에 복사할 내용 ---")
        print('[')
        for i, cap in enumerate(generated_captions):
            print(f'  "{cap}"{"," if i < len(generated_captions) - 1 else ""}')
        print(']')
        
        return generated_captions
    except Exception as e:
        print(f"캡션 생성 중 오류 발생: {e}")
        return []

## 4. 이미지-텍스트 유사도 검증

In [ ]:
def verify_image_text_similarity(user_image, gt_captions, threshold_low=0.5, threshold_high=0.8):
    """
    이미지와 텍스트 캡션 간의 유사도를 계산하여 검증
    
    Args:
        user_image: PIL Image 객체
        gt_captions: Ground Truth 캡션 리스트
        threshold_low: 낮은 임계값
        threshold_high: 높은 임계값
    
    Returns:
        best_score: 최고 매칭 점수 (0~1)
        feedback: 피드백 메시지
    """
    # 모델 로드
    mat_processor, mat_model = load_model("matching")
    
    if not gt_captions:
        return 0.0, "오류: Ground Truth 캡션이 비어있습니다."
    
    match_scores = []
    
    print("\n[1. 이미지-텍스트 유사도 검증]")
    print("AI가 사진을 분석 중입니다...")
    
    try:
        for gt_caption in gt_captions:
            # 이미지와 정답 캡션을 모델이 이해하도록 처리
            inputs = mat_processor(images=user_image, text=gt_caption, return_tensors="pt").to(device)
            
            # ITC 로짓(코사인 유사도 기반 점수)을 텐서로 가져옴
            cosine_score = mat_model(**inputs, use_itm_head=False)[0]
            
            # sigmoid 함수를 적용하여 [0, 1] 확률로 변환
            prob_score_tensor = torch.sigmoid(cosine_score)
            prob_score_float = prob_score_tensor.item()
            
            match_scores.append(prob_score_float)
        
        # 가장 높은 매칭 점수를 최종 점수로 사용
        best_score = max(match_scores)
        
        print(f"\n[AI 분석 결과] 최고 매칭 점수: {best_score*100:.2f}점")
        
        # 피드백 생성
        if best_score > threshold_high:
            feedback = "🎉 정답입니다! 미션 성공!"
        elif best_score > threshold_low:
            feedback = f"🤔 오! 거의 근접했어요! (점수: {best_score*100:.2f}점)"
        else:
            feedback = f"아쉽네요. 힌트와는 조금 다른 것 같아요. (점수: {best_score*100:.2f}점)"
        
        return best_score, feedback
    except Exception as e:
        print(f"이미지-텍스트 유사도 검증 중 오류 발생: {e}")
        return 0.0, f"오류: {str(e)}"

## 5. 텍스트-텍스트 유사도 검증

In [ ]:
def verify_text_text_similarity(user_image, gt_captions, threshold_low=0.5, threshold_high=0.8):
    """
    사용자 이미지에서 캡션을 생성한 후, Ground Truth 캡션들과 텍스트 유사도를 비교
    
    Args:
        user_image: PIL Image 객체
        gt_captions: Ground Truth 캡션 리스트
        threshold_low: 낮은 임계값
        threshold_high: 높은 임계값
    
    Returns:
        best_score: 최고 유사도 점수 (0~1)
        user_caption: 생성된 사용자 이미지 캡션
        feedback: 피드백 메시지
    """
    # 모델 로드
    cap_processor, cap_model = load_model("captioning")
    sbert_model = load_sbert_model()
    
    if not gt_captions:
        return 0.0, "", "오류: Ground Truth 캡션이 비어있습니다."
    
    print("\n[2. 텍스트-텍스트 유사도 검증]")
    print("AI가 사진 캡션을 생성 중입니다...")
    
    try:
        # 사용자 사진으로 캡션 생성
        inputs_cap = cap_processor(images=user_image, return_tensors="pt").to(device)
        outputs_cap = cap_model.generate(**inputs_cap, max_length=50)
        user_caption = cap_processor.decode(outputs_cap[0], skip_special_tokens=True)
        print(f"  [생성된 캡션]: {user_caption}")
        
        print("생성된 캡션과 정답 캡션들의 유사도를 비교합니다...")
        
        # 텍스트 임베딩 생성 (SBERT)
        user_embedding = sbert_model.encode([user_caption])
        gt_embeddings = sbert_model.encode(gt_captions)
        
        # 코사인 유사도 계산
        sim_scores = cosine_similarity(user_embedding, gt_embeddings)
        best_score = np.max(sim_scores)
        
        print(f"\n[AI 분석 결과] 최고 캡션 유사도: {best_score*100:.2f}점")
        
        # 피드백 생성
        if best_score > threshold_high:
            feedback = "🎉 정답입니다! 미션 성공!"
        elif best_score > threshold_low:
            feedback = f"🤔 오! 거의 근접했어요! (점수: {best_score*100:.2f}점)"
        else:
            feedback = f"아쉽네요. 힌트와는 조금 다른 것 같아요. (점수: {best_score*100:.2f}점)"
        
        return best_score, user_caption, feedback
    except Exception as e:
        print(f"텍스트-텍스트 유사도 검증 중 오류 발생: {e}")
        return 0.0, "", f"오류: {str(e)}"


## 6. 질문 세트 정리

In [ ]:
# ============================================================================
# 6. 질문 세트 정리
# ============================================================================

# 랜드마크별 질문 세트 정의
QUESTIONS = {
    "피노키오": [
        "Is there a prominent human-shaped statue in the picture?",
        "If there is a statue, is it a character from a fairy tale?",
        "Does the statue have a particularly long nose?",
        "Is the statue wearing green-colored clothes?",
        "Are there flower patterns on the statue's clothes?",
        "Is the statue holding something?",
        "Is the object the statue is holding a book?",
        "Is the hat the statue is wearing a pointed cone shape?",
        "Are the statue's arms relatively thin and long?",
        "Is the statue holding up a finger with its right hand, like the number 1?"
    ],
    "네모탑": [
        "Is this object shaped like a tower?",
        "Is this tower shaped like stacked square boxes?",
        "Is the height much longer than the width?",
        "Is this object made of wood?",
        "Is the color reddish-brown?",
        "Does the color look like rusty metal?",
        "Is the surface smooth?",
        "Does the sculpture only use dark colors?",
        "Does this sculpture have many layers?",
        "Is the number of layers exactly 5?",
        "Do the layers get smaller as they go up?",
        "Is the bottom part the widest?",
        "Does each layer have a square opening on the front?",
        "Does the height of this sculpture look more than twice a person's height?"
    ],
    "지혜의숲 조각상": [
        "Is this picture taken inside a building?",
        "Does the background show a place with many books?",
        "Do wooden bookshelves cover the walls?",
        "Does the room have a high ceiling?",
        "Is the floor made of wood?",
        "Is the main object a human-shaped sculpture?",
        "Is the sculpture sitting down?",
        "Does the sculpture have its hands near its face?",
        "Is the sculpture holding binoculars?",
        "Is the sculpture looking straight ahead?",
        "Does the sculpture wear a suit?",
        "Is the suit covered in many bright colors?",
        "Does the suit have patterns like flowers?",
        "Does the sculpture's suit have a lot of green and yellow?",
        "Is the sculpture's face also painted with patterns?",
        "Is the sculpture sitting on a square base?",
        "Does the base look like it is made of stacked small blocks?",
        "Does the base have letters written on it?",
        "Is a small container placed next to the sculpture?",
        "Does that small container have patterns on it too?"
    ]
}

## 7. VQA 유사도 검증 (질문 기반 YES 비율로 판정)

In [ ]:
# ============================================================================
# 7. VQA 유사도 검증 (질문 기반 YES 비율로 판정)
# ============================================================================

def verify_vqa_similarity(user_image, landmark_name, required_yes_ratio=0.75):
    """
    VQA 모델을 사용하여 질문-답변 기반으로 랜드마크 인식 검증
    Ground Truth 캡션이 아닌 질문에 대한 YES 비율로 판정
    
    Args:
        user_image: PIL Image 객체
        landmark_name: 랜드마크 이름 (QUESTIONS 딕셔너리의 키)
        required_yes_ratio: 정답으로 판정할 YES 비율 (기본값: 0.75)
    
    Returns:
        yes_ratio: YES 답변 비율 (0~1)
        feedback: 피드백 메시지
    """
    # 모델 로드
    vqa_processor, vqa_model = load_model("vqa")
    
    print("\n[3. VQA 유사도 검증]")
    print("AI가 사진을 분석 중입니다...")
    
    if landmark_name not in QUESTIONS:
        return None, f"오류: '{landmark_name}'는 정의되지 않은 랜드마크 이름입니다."
    
    landmark_questions = QUESTIONS[landmark_name]
    yes_count = 0
    total_questions = len(landmark_questions)
    
    if total_questions == 0:
        return None, f"경고: '{landmark_name}'에 대한 질문이 정의되지 않았습니다."
    
    try:
        for question in tqdm(landmark_questions, desc=f"VQA 질문 처리 중"):
            try:
                # VQA 모델에 이미지와 질문 전달
                inputs = vqa_processor(user_image, question, return_tensors="pt").to(device)
                
                # 답변 생성 (max_length를 짧게 하여 단답형 유도)
                out = vqa_model.generate(**inputs, max_length=20)
                answer = vqa_processor.decode(out[0], skip_special_tokens=True).strip().lower()
                
                # 'yes' 개수 세기
                if answer == 'yes':
                    yes_count += 1
            except Exception as e:
                print(f"\n질문 처리 중 오류: {e}")
        
        # YES 비율 계산
        yes_ratio = (yes_count / total_questions) if total_questions > 0 else 0
        print(f"\n총 질문 수: {total_questions}, 'yes' 답변 수: {yes_count}")
        print(f"YES 비율: {yes_ratio*100:.2f}%")
        
        # 결과 판정 (YES 비율 기준)
        if yes_ratio >= required_yes_ratio:
            feedback = f"🎉 정답입니다! 입력된 사진은 '{landmark_name}'일 가능성이 높습니다. (YES 비율: {yes_ratio*100:.2f}%)"
        else:
            feedback = f"🤔 아쉽네요. 입력된 사진은 '{landmark_name}'와 일치하지 않는 것 같습니다. (YES 비율: {yes_ratio*100:.2f}%)"
        
        return yes_ratio, feedback
    except Exception as e:
        print(f"VQA 검증 중 오류 발생: {e}")
        return None, f"오류: {str(e)}"

## 8. 미션 시스템 (argument로 검증 방식 선택)

In [ ]:
# ============================================================================
# 8. 미션 시스템 (argument로 검증 방식 선택)
# ============================================================================

def run_mission_verification(verification_method="image_text", landmark_name=None, mission_data=None, 
                             image_path=None, data_dir=None):
    """
    미션 검증 시스템 실행
    
    Args:
        verification_method: 검증 방법 선택
            - "image_text": 이미지-텍스트 유사도 검증 (Ground Truth 캡션 필요)
            - "text_text": 텍스트-텍스트 유사도 검증 (Ground Truth 캡션 필요)
            - "vqa": VQA 유사도 검증 (질문 세트만 필요, Ground Truth 캡션 불필요)
        landmark_name: 랜드마크 이름 (VQA 검증 시 필수, 다른 검증 시 Ground Truth 생성에 사용)
        mission_data: 미션 데이터 딕셔너리 (None이면 기본값 사용)
        image_path: 이미지 파일 경로 (로컬 환경용, None이면 업로드 사용)
        data_dir: 데이터 디렉토리 경로 (None이면 기본값 사용)
    """
    # 미션 데이터 설정
    if mission_data is None:
        mission_data = {
            "id": 1,
            "daily_keyword": "지식",
            "landmark_name": "지혜의숲 조각상",
            "hint_text_1": "수만 권의 이야기가 쌓인 시간의 흔적을 찾아보세요.",
            "hint_text_2": "높은 서가들이 숲처럼 우거진 곳입니다.",
            "hint_text_3": "ㅈㅎㅇ ㅅ (초성 힌트)",
            "score_threshold_low": 0.5,
            "score_threshold_high": 0.8,
            "required_yes_ratio": 0.75  # VQA 검증용
        }
    
    # 랜드마크 이름 설정
    if landmark_name is None:
        landmark_name = mission_data.get("landmark_name", "지혜의숲 조각상")
    
    print(f"\n{'='*60}")
    print(f"오늘의 미션: #{mission_data['daily_keyword']}")
    print(f"랜드마크: {landmark_name}")
    print(f"검증 방식: {verification_method}")
    print(f"{'='*60}")
    print(f"\n힌트: {mission_data['hint_text_1']}")
    
    # 이미지 로드
    try:
        if image_path:
            # 로컬 파일 경로가 제공된 경우
            print(f"\n이미지를 로드합니다: {image_path}")
            user_image = Image.open(image_path).convert("RGB")
            file_name = os.path.basename(image_path)
        elif IN_COLAB:
            # Colab 환경: 파일 업로드
            print("\n정답이라고 생각하는 사진을 업로드해주세요:")
            uploaded = files.upload()
            
            if len(uploaded) == 0:
                print("업로드된 파일이 없습니다. 셀을 다시 실행해주세요.")
                return
            
            file_name = list(uploaded.keys())[0]
            user_image = Image.open(io.BytesIO(uploaded[file_name])).convert("RGB")
        else:
            print("오류: 이미지 경로를 제공해야 합니다.")
            print("사용법: run_mission_verification(..., image_path='your_image.jpg')")
            return
        
        print(f"\n[업로드한 사진: {file_name}]")
        
        # 이미지 표시 (환경에 따라)
        if IN_COLAB:
            try:
                from IPython.display import display as ipy_display
                ipy_display(user_image.resize((300, 300)))
            except:
                print("이미지 표시 기능을 사용할 수 없습니다.")
        else:
            print(f"이미지 크기: {user_image.size}")
            
    except Exception as e:
        print(f"이미지 로드 중 오류 발생: {e}")
        return
    
    # 검증 방법에 따라 실행
    if verification_method == "image_text":
        # Ground Truth 캡션 생성
        print("\n[Ground Truth 캡션 생성 중...]")
        gt_captions = generate_ground_truth_captions_from_folder(landmark_name, data_dir)
        
        if not gt_captions:
            print("경고: Ground Truth 캡션을 생성할 수 없습니다.")
            return
        
        # 이미지-텍스트 유사도 검증
        best_score, feedback = verify_image_text_similarity(
            user_image,
            gt_captions,
            mission_data['score_threshold_low'],
            mission_data['score_threshold_high']
        )
        print(f"\n{feedback}")
        
        # 힌트 제공
        if best_score > mission_data['score_threshold_low'] and best_score <= mission_data['score_threshold_high']:
            print(f"💡 힌트 2: {mission_data['hint_text_2']}")
        elif best_score <= mission_data['score_threshold_low']:
            print(f"💡 최종 힌트: {mission_data['hint_text_3']}")
        else:
            print("(보상: 100 마일리지 적립 💰)")
    
    elif verification_method == "text_text":
        # Ground Truth 캡션 생성
        print("\n[Ground Truth 캡션 생성 중...]")
        gt_captions = generate_ground_truth_captions_from_folder(landmark_name, data_dir)
        
        if not gt_captions:
            print("경고: Ground Truth 캡션을 생성할 수 없습니다.")
            return
        
        # 텍스트-텍스트 유사도 검증
        best_score, user_caption, feedback = verify_text_text_similarity(
            user_image,
            gt_captions,
            mission_data['score_threshold_low'],
            mission_data['score_threshold_high']
        )
        print(f"\n{feedback}")
        
        # 힌트 제공
        if best_score > mission_data['score_threshold_low'] and best_score <= mission_data['score_threshold_high']:
            print(f"💡 힌트 2: {mission_data['hint_text_2']}")
        elif best_score <= mission_data['score_threshold_low']:
            print(f"💡 최종 힌트: {mission_data['hint_text_3']}")
        else:
            print("(보상: 100 마일리지 적립 💰)")
    
    elif verification_method == "vqa":
        # VQA 검증 (Ground Truth 캡션 불필요, 질문 세트만 사용)
        yes_ratio, feedback = verify_vqa_similarity(
            user_image,
            landmark_name,
            mission_data.get('required_yes_ratio', 0.75)
        )
        print(f"\n{feedback}")
        
        # 힌트 제공
        if yes_ratio is not None:
            if yes_ratio >= mission_data.get('required_yes_ratio', 0.75):
                print("(보상: 100 마일리지 적립 💰)")
            elif yes_ratio >= 0.5:  # 중간 정도
                print(f"💡 힌트 2: {mission_data['hint_text_2']}")
            else:
                print(f"💡 최종 힌트: {mission_data['hint_text_3']}")
    
    else:
        print(f"알 수 없는 검증 방법: {verification_method}")
        print("사용 가능한 방법: 'image_text', 'text_text', 'vqa'")

## 9. 유틸리티 함수

In [ ]:
# ============================================================================
# 9. 유틸리티 함수
# ============================================================================

def get_available_landmarks():
    """사용 가능한 랜드마크 목록 반환"""
    return list(QUESTIONS.keys())

def print_system_info():
    """시스템 정보 출력"""
    print("=" * 60)
    print("BLIP 기반 랜드마크 이미지 검증 시스템")
    print("=" * 60)
    print(f"\n환경: {'Google Colab' if IN_COLAB else '로컬'}")
    print(f"장치: {device}")
    print(f"데이터 디렉토리: {DATA_DIR}")
    print(f"\n사용 가능한 랜드마크:")
    for i, landmark in enumerate(get_available_landmarks(), 1):
        print(f"  {i}. {landmark}")
    print("\n사용 가능한 검증 방법:")
    print("  1. image_text: 이미지-텍스트 유사도 검증 (Ground Truth 캡션 필요)")
    print("  2. text_text: 텍스트-텍스트 유사도 검증 (Ground Truth 캡션 필요)")
    print("  3. vqa: VQA 유사도 검증 (질문 세트만 필요)")
    
    if IN_COLAB:
        print("\n사용 예시 (Colab):")
        print("  run_mission_verification('image_text', '지혜의숲 조각상')")
        print("  run_mission_verification('vqa', '피노키오')")
    else:
        print("\n사용 예시 (로컬):")
        print("  run_mission_verification('image_text', '지혜의숲 조각상', image_path='test.jpg')")
        print("  run_mission_verification('vqa', '피노키오', image_path='pinocchio.jpg')")
    print("=" * 60)

In [ ]:
print_system_info()

In [ ]:
# ============================================================================
# 예시 1: 시스템 정보 확인
# ============================================================================

def example_1_system_info():
    """시스템 정보 및 사용 가능한 랜드마크 확인"""
    print("=" * 60)
    print("예시 1: 시스템 정보 확인")
    print("=" * 60)
    
    # 시스템 정보 출력
    print_system_info()
    
    # 사용 가능한 랜드마크 목록
    landmarks = get_available_landmarks()
    print(f"\n사용 가능한 랜드마크: {landmarks}")


# ============================================================================
# 예시 2: VQA 검증 (가장 간단)
# ============================================================================

def example_2_vqa_verification():
    """VQA 검증 - Ground Truth 캡션 없이 질문-답변으로 검증"""
    print("\n" + "=" * 60)
    print("예시 2: VQA 검증")
    print("=" * 60)
    
    # Google Colab 환경
    # run_mission_verification('vqa', '피노키오')
    
    # 로컬 환경 (이미지 경로 지정)
    run_mission_verification(
        verification_method='vqa',
        landmark_name='피노키오',
        image_path='test_pinocchio.jpg'  # 실제 파일 경로로 변경
    )


# ============================================================================
# 예시 3: 이미지-텍스트 유사도 검증
# ============================================================================

def example_3_image_text_verification():
    """이미지-텍스트 유사도 검증"""
    print("\n" + "=" * 60)
    print("예시 3: 이미지-텍스트 유사도 검증")
    print("=" * 60)
    
    run_mission_verification(
        verification_method='image_text',
        landmark_name='지혜의숲 조각상',
        image_path='test_wisdom_forest.jpg'  # 실제 파일 경로로 변경
    )


# ============================================================================
# 예시 4: 텍스트-텍스트 유사도 검증
# ============================================================================

def example_4_text_text_verification():
    """텍스트-텍스트 유사도 검증"""
    print("\n" + "=" * 60)
    print("예시 4: 텍스트-텍스트 유사도 검증")
    print("=" * 60)
    
    run_mission_verification(
        verification_method='text_text',
        landmark_name='네모탑',
        image_path='test_nemotop.jpg'  # 실제 파일 경로로 변경
    )


# ============================================================================
# 예시 5: Ground Truth 캡션 생성
# ============================================================================

def example_5_generate_captions():
    """폴더의 모든 이미지에서 Ground Truth 캡션 생성"""
    print("\n" + "=" * 60)
    print("예시 5: Ground Truth 캡션 생성")
    print("=" * 60)
    
    captions = generate_ground_truth_captions_from_folder(
        landmark_name='피노키오',
        data_dir='./data'  # 데이터 디렉토리 경로
    )
    
    print(f"\n생성된 캡션 수: {len(captions)}")
    print("\n생성된 캡션 목록:")
    for i, caption in enumerate(captions[:5], 1):  # 처음 5개만 출력
        print(f"  {i}. {caption}")


# ============================================================================
# 예시 6: 단일 이미지 캡션 생성
# ============================================================================

def example_6_single_image_caption():
    """단일 이미지에서 캡션 생성 (JSON 형식 출력)"""
    print("\n" + "=" * 60)
    print("예시 6: 단일 이미지 캡션 생성")
    print("=" * 60)
    
    # 이미지 로드
    image = Image.open('test_image.jpg')  # 실제 파일 경로로 변경
    
    # 캡션 생성 (JSON 형식으로 출력됨)
    captions = generate_single_image_caption(image)
    
    print(f"\n생성된 캡션: {captions}")


# ============================================================================
# 예시 7: 커스텀 미션 데이터
# ============================================================================

def example_7_custom_mission():
    """커스텀 미션 데이터로 검증"""
    print("\n" + "=" * 60)
    print("예시 7: 커스텀 미션 데이터")
    print("=" * 60)
    
    # 커스텀 미션 데이터
    custom_mission = {
        "id": 99,
        "daily_keyword": "문화",
        "landmark_name": "피노키오",
        "hint_text_1": "이야기 속 주인공을 찾아보세요",
        "hint_text_2": "긴 코가 특징인 캐릭터입니다",
        "hint_text_3": "ㅍㄴㅋㅇ (초성 힌트)",
        "score_threshold_low": 0.4,
        "score_threshold_high": 0.75,
        "required_yes_ratio": 0.7
    }
    
    run_mission_verification(
        verification_method='vqa',
        landmark_name='피노키오',
        mission_data=custom_mission,
        image_path='test_pinocchio.jpg'
    )


# ============================================================================
# 예시 8: 커스텀 데이터 디렉토리
# ============================================================================

def example_8_custom_data_directory():
    """커스텀 데이터 디렉토리 사용"""
    print("\n" + "=" * 60)
    print("예시 8: 커스텀 데이터 디렉토리")
    print("=" * 60)
    
    run_mission_verification(
        verification_method='vqa',
        landmark_name='피노키오',
        image_path='test_pinocchio.jpg',
        data_dir='./my_custom_data'  # 커스텀 데이터 경로
    )


# ============================================================================
# 예시 9: 개별 검증 함수 직접 사용
# ============================================================================

def example_9_direct_verification():
    """검증 함수를 직접 호출"""
    print("\n" + "=" * 60)
    print("예시 9: 개별 검증 함수 직접 사용")
    print("=" * 60)
    
    # 이미지 로드
    user_image = Image.open('test_pinocchio.jpg')
    
    # VQA 검증
    yes_ratio, feedback = verify_vqa_similarity(
        user_image=user_image,
        landmark_name='피노키오',
        required_yes_ratio=0.75
    )
    
    print(f"\nYES 비율: {yes_ratio}")
    print(f"피드백: {feedback}")


# ============================================================================
# 예시 10: 배치 처리
# ============================================================================

def example_10_batch_verification():
    """여러 이미지를 순차적으로 검증"""
    print("\n" + "=" * 60)
    print("예시 10: 배치 처리")
    print("=" * 60)
    
    test_images = [
        ('피노키오', 'test_pinocchio_1.jpg'),
        ('피노키오', 'test_pinocchio_2.jpg'),
        ('네모탑', 'test_nemotop_1.jpg'),
        ('지혜의숲 조각상', 'test_wisdom_1.jpg')
    ]
    
    results = []
    
    for landmark_name, image_path in test_images:
        print(f"\n검증 중: {image_path}")
        
        try:
            image = Image.open(image_path)
            yes_ratio, feedback = verify_vqa_similarity(
                user_image=image,
                landmark_name=landmark_name,
                required_yes_ratio=0.75
            )
            
            results.append({
                'landmark': landmark_name,
                'image': image_path,
                'yes_ratio': yes_ratio,
                'success': yes_ratio >= 0.75 if yes_ratio else False
            })
        except Exception as e:
            print(f"오류: {e}")
            results.append({
                'landmark': landmark_name,
                'image': image_path,
                'error': str(e),
                'success': False
            })
    
    # 결과 요약
    print("\n" + "=" * 60)
    print("배치 처리 결과 요약")
    print("=" * 60)
    for result in results:
        status = "✅ 성공" if result.get('success') else "❌ 실패"
        print(f"{status} | {result['landmark']} | {result['image']}")


# ============================================================================
# 예시 11: 모든 검증 방법 비교
# ============================================================================

def example_11_compare_all_methods():
    """하나의 이미지로 세 가지 검증 방법 비교"""
    print("\n" + "=" * 60)
    print("예시 11: 모든 검증 방법 비교")
    print("=" * 60)
    
    image_path = 'test_pinocchio.jpg'
    landmark_name = '피노키오'
    
    # 1. 이미지-텍스트 유사도
    print("\n1️⃣ 이미지-텍스트 유사도 검증")
    print("-" * 40)
    run_mission_verification('image_text', landmark_name, image_path=image_path)
    
    # 2. 텍스트-텍스트 유사도
    print("\n2️⃣ 텍스트-텍스트 유사도 검증")
    print("-" * 40)
    run_mission_verification('text_text', landmark_name, image_path=image_path)
    
    # 3. VQA 검증
    print("\n3️⃣ VQA 검증")
    print("-" * 40)
    run_mission_verification('vqa', landmark_name, image_path=image_path)


# BLIP 랜드마크 검증 시스템

In [ ]:
print("BLIP 랜드마크 검증 시스템 - 사용 예시")
    print("=" * 60)
    print("\n실행할 예시를 선택하세요:")
    print("  1. 시스템 정보 확인")
    print("  2. VQA 검증")
    print("  3. 이미지-텍스트 유사도 검증")
    print("  4. 텍스트-텍스트 유사도 검증")
    print("  5. Ground Truth 캡션 생성")
    print("  6. 단일 이미지 캡션 생성")
    print("  7. 커스텀 미션 데이터")
    print("  8. 커스텀 데이터 디렉토리")
    print("  9. 개별 검증 함수 직접 사용")
    print(" 10. 배치 처리")
    print(" 11. 모든 검증 방법 비교")
    print(" 0. 모두 실행")
    
    choice = input("\n선택 (0-11): ").strip()
    
    if choice == "1":
        example_1_system_info()
    elif choice == "2":
        example_2_vqa_verification()
    elif choice == "3":
        example_3_image_text_verification()
    elif choice == "4":
        example_4_text_text_verification()
    elif choice == "5":
        example_5_generate_captions()
    elif choice == "6":
        example_6_single_image_caption()
    elif choice == "7":
        example_7_custom_mission()
    elif choice == "8":
        example_8_custom_data_directory()
    elif choice == "9":
        example_9_direct_verification()
    elif choice == "10":
        example_10_batch_verification()
    elif choice == "11":
        example_11_compare_all_methods()
    elif choice == "0":
        # 모두 실행 (시간이 오래 걸릴 수 있음)
        example_1_system_info()
        # 나머지는 실제 이미지가 필요하므로 건너뜀
        print("\n나머지 예시는 실제 이미지 파일이 필요합니다.")
    else:
        print("잘못된 선택입니다.")